# <h1><center>Simulation of *Phaedoactylum tricronutum* growth in a Photobioreactor (PBR)</center></h1> 

***
This notebook refers to the base code of the publication "**Towards Multiscale Modeling to Predict Diatom Metabolites Production for Biofuels and High-Value Compounds**". 

**Please cite as:** BRANCO-VIEIRA, M, CAETANO, N.S., LIMA, A.R.J., TÖPFER, N. Towards Multiscale Modeling to Predict Diatom Metabolites Production for Biofuels and High-Value Compounds. In: Caetano, N.S., Felgueiras, M.C. (eds) The 9th International Conference on Energy and Environment Research. ICEER 2022. Environmental Science and Engineering. Springer (2023). [![DOI:10.1007/978-3-031-43559-1_31](https://zenodo.org/badge/DOI/10.1007/978-3-031-43559-1_31.svg)](https://link.springer.com/chapter/10.1007/978-3-031-43559-1_31)

The aim is to analyze the growth of P. tricornutum in a bubble column PRB, utilizing the experimental results published in <a href="https://doi.org/10.1016/j.fuel.2020.117357" target="_blank">Branco-Vieira et al (2020)</a>.
 
***

The simulation encompasses the initial steps to upscale the growth of P. tricornutum from the cell to the PBR unit.


#### Pipeline:

**A- Integrating experimental dataset**
1. Update the medium composition
2. Update the pigment composition
3. Update the biomass composition

**B- Implementing the constraints**
1. Global constraints
2. Photons uptake constraints
3. Photobioreactor constraints 
4. Light constraints

P. tricornutum model: <a href="https://doi.org/10.1111/nph.15685" target="_blank">iLB1034</a>.



Python 3.12 Anaconda 3 - Cobra 0.29.0  -->see requerements.txt or use the .yml file to set the virtual environment. 

Author: Monique Branco [![GitHub](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/moniquebranco) 

Created: 03/11/2021,
Updated: 09/10/2023

Disclaimer: Part of the code was adapted from: <a href="https://doi.org/10.1111/nph.15685" target="_blank">Broddrick, et al (2019)</a>. 


In [2]:
import pandas as pd
import numpy as np
import csv
import cobra
import xlrd
import re
import math as m
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
import copy
import warnings
warnings.filterwarnings('ignore')

### Load in the model

In [3]:
# Load in the model
model = cobra.io.load_json_model('Model_iLB1034.json')

### A) Integrating the experimental dataset

### A1) Update the media components
##### Culture Media: Guillard’s f/2 formulation 



In [4]:
# Load the media composition

EXrxn = ['EX_co2_e','EX_hco3_e','EX_no2_e','EX_no3_e','EX_nh4_e','EX_biotin_e','EX_fe2_e','EX_h_e','EX_h2o_e','EX_o2_e','EX_pi_e','EX_na1_e','EX_so4_e','EX_hso3_e','EX_mg2_e','EX_glyclt_e','EX_selt_e','EX_glc_e','EX_cl_e','EX_thm_e','EX_h2_e','EX_fol_e','EX_co_e','EX_cyan_e','EX_cynt_e','EX_tcynt_e','EX_lac_d_e','EX_etoh_e']
EXub = [1000,1000,0,0,1000,0,1000,1000,1000,1000,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
EXlb = [0,-1000,0,-1000,0,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,0,-1000,0,0,0,-1000,-1000,0,0,0,0,0,0,0,0]
for i in range(len(EXrxn)):
    rx = EXrxn[i]
    model.reactions.get_by_id(rx).lower_bound = EXlb[i]
    model.reactions.get_by_id(rx).upper_bound = EXub[i]

### A2) Update the pigment composition 

The pigment composition was integrated using the normalized values of pigment content percentages. These values correspond to the biochemical characterization after 14 days of cultivation.

#### Output: pigment composition integration into the reaction "biomass_pigm_h"

In [5]:
pigment_mass = {'caro_h':0.0340, #beta-carotene  
               'cholphya_h':0.7691, #chlorophyll a
               'cholphyc1_h':0.0130, #chlorophyll c1
               'cholphyc2_h':0.0287, #chlorophyll c2
               'fxanth_h':0.1552} #fucoxanthin
pigment_data = dict()

#pigment mass to normalized values based on molecular weight
for pigm,mass in pigment_mass.items():
    met_obj = model.metabolites.get_by_id(pigm)
    met_stoich = mass/met_obj.formula_weight
    pigment_data[met_obj]=met_stoich*-1*1000. #the fluxes in the model are in mmol
    
r = model.reactions.biomass_pigm_h

new_stoich = pigment_data

# now have a dictionary of new stoichs 
for m,s in r.metabolites.items():
    r.add_metabolites(new_stoich)

# Then get the total to equal 1 mg biomass DW
total = 0
for m,s in r.metabolites.items():
    gfw = model.metabolites.get_by_id(m.id).formula_weight
    mass = gfw*s*-1
    total = total+mass
correction = total/1000 # this will get it to 1000 ug total mass

# Then adjust the stoichiometry as appropriate
for m,s in r.metabolites.items(): # now change the stoich
    to_add = s/correction-s
    r.add_metabolites({m:to_add})

# Finally build the biomass_c metabolite
    imbal = r.check_mass_balance()
    if 'charge' in imbal.keys():
        met_charge = imbal['charge']*-1
        del imbal['charge']
    met_mass = 0
    formula_string = ''
    for e,v in imbal.items():
        if v > 1e-10 or v < -1e-10:
            mass = elements_and_molecular_weights[e]
            met_mass = met_mass+(mass*-1*v)
            form_str = e+str(-1*v)
            formula_string = formula_string + form_str

met = model.metabolites.biomass_pigm_h # the pigments components were included into a new metabolite, to facilitate further modeling
met.formula = formula_string
met.charge = met_charge
r.add_metabolites({met:1})

model.repair()

### A3) Update the biomass composition 

The biomass components were integrated using the normalized values of macromolecular components in percentage. 

#### Output: biomass composition integration as a metabolite "biomass_c"

In [6]:
bof_ratios = {'biomass_carb_c':0.1568, #carbohydrates
               'biomass_dna_c':0.0082, #DNA
               'biomass_mem_lipids_c':0.1069, #membrane lipids
               'biomass_pigm_h':0.0832, #pigments
               'biomass_pro_c':0.5797, #proteins
               'biomass_rna_c':0.0653} #RNA
bof_data = dict()

for bofcmp,percent in bof_ratios.items():
    met_obj = model.metabolites.get_by_id(bofcmp)
    bof_data[met_obj]=percent*-1
    
# copy the bof and change to carbon storage
r = model.reactions.bof_c
new_stoich = bof_data

# now have a dictionary of new stoichs for the model
for m,s in r.metabolites.items():
    stoich = s*-1
    temp_dict = {m:stoich}
    r.add_metabolites(temp_dict)
r.add_metabolites(new_stoich)

# Then get the total to equal 1 mg biomass DW
total = 0
for m,s in r.metabolites.items():
    gfw = model.metabolites.get_by_id(m.id).formula_weight
    mass = gfw*s*-1
    total = total+mass
correction = total/1000 # this will get it to 1000 ug total mass

# Then adjust the stoichiometry as appropriate
for m,s in r.metabolites.items(): # now change the stoich
    to_add = s/correction-s
    r.add_metabolites({m:to_add})

# Finally build the biomass_c metabolite
imbal = r.check_mass_balance()
if 'charge' in imbal.keys():
    met_charge = imbal['charge']*-1
    del imbal['charge']
met_mass = 0
formula_string = ''
for e,v in imbal.items():
    if v > 1e-12 or v < -1e-12:
        mass = elements_and_molecular_weights[e]
        met_mass = met_mass+(mass*-1*v)
        form_str = e+str(-1*v)
        formula_string = formula_string + form_str

# make a new biomass metabolite
met = model.metabolites.biomass_c
met.formula = formula_string
met.charge = met_charge
r.add_metabolites({met:1})

# Add GAM constraint
gam_met = model.metabolites.GAM_const_c
r.add_metabolites({gam_met:1})


model.repair()

### B- Implementing the constraints

### B1-Global Constraints 

#### The global constraints refer to those suggested by the original iLB1034 model's authors.

#### Output: turn-off reactions that cause unrealistic loops, set CEF constrains, changing bounds to umol and set the energetic coupling.


In [7]:
# Turn off reactions that cause biologically unrealistic flux loops 

rxn =  ['DIATOXEPOX_h','VIOXANDE_h','ANTHXANDE_h']
for i in range(len(rxn)):
    rx = rxn[i]
    model.reactions.get_by_id(rx).lower_bound = 0
    model.reactions.get_by_id(rx).upper_bound = 0
    
# flux only constraints, cause an unrealistic loop in a cofactor metabolic pathway
model.reactions.PDYXPT_c.upper_bound = 0.
model.reactions.PDYXPT_c.lower_bound = 0.
model.reactions.PYDXDH_c.upper_bound = 0.
model.reactions.PYDXDH_c.lower_bound = 0.
    

# remove constraints on CEF_h, is calculated in the simulation

model.reactions.CEF_h.lower_bound = 0.0
model.reactions.CEF_h.upper_bound = 1000

# Changing the bounds to umol instead of mmol (easier to integrate photon constrains)

for r in model.reactions:
    r.lower_bound = r.lower_bound*10.
    r.upper_bound = r.upper_bound*10.
    
    
# Energetic coupling as in Ballieu 2015 Nature paper
# Identical to the original model as suggested by the authors

from cobra import Metabolite
m1 = Metabolite(
    'mito_coupling_const_h',
    formula='',
    name='Mito-Plastid Energetic Coupling Constraint',
    compartment='h')

m1._constraint_sense = 'G'

reaction = model.reactions.PSI_u
reaction.add_metabolites({m1:-0.15}) #penalty of 15% in PSI

reaction = model.reactions.NADHOR_m
reaction.add_metabolites({m1:1.})

# Non-photochemical loss of photons
reaction = model.reactions.DM_biomass_c.copy()
reaction.id = 'DM_photon_c'
reaction.name = 'Demand Reaction: Photon'
m1 = model.metabolites.biomass_c
m2 = model.metabolites.photon_c

reaction.add_metabolites({m1:1,
                         m2:-1})

model.add_reactions([reaction])
model.repair()

#Temp demand reaction to allow excess photons to leave the system
reaction = model.reactions.DM_biomass_c.copy()
reaction.id = 'DM_pigm'
reaction.name = 'Demand Reaction: Pigm'
m1 = model.metabolites.biomass_c
m2 = model.metabolites.biomass_pigm_h

reaction.add_metabolites({m1:1,
                         m2:-1})

reaction.reaction
model.add_reactions([reaction])
model.repair()

model.reactions.DM_biomass_c_acc_c.upper_bound = 1000000
model.reactions.DM_biomass_c_acc_c.lower_bound = 0
# model.reactions.DM_pigm.upper_bound = 0
# model.reactions.DM_pigm.lower_bound = 0

### B2- Photon uptake constraints

"The photon absorption flux was modeled in the PAR spectral range (400–700 nm) using the method published in (Broddrick et al., 2019b). Briefly, the photon uptake was modeled in terms of biomass-normalized photon absorption rate in μmol photons dry weight (DW)−1 h−1 (BPA), considering the experimentally measured concentration of chlorophyll-a (mgChla), the experimental biomass dry weight in grams (gDW), the solar irradiance in μmol photons m-2 s-1 (Io) and the Chla normalized optical absorption cross-section (Chla*) in cm2 mg chlorophyll-a−1"

                                 BPA = mgChla/gDW x Io x chla* 

#### Chla* will be calculate as:

chla* = (2.303 x (Absorption espectra of Chla))/mgChla 


#### Output: values for chla* in m2/mgChla; 

In [8]:
# Photon constraints

# 1) Photon absorption coefficient 'chla*' where:

# Absorbed photons (BPA) = mgChla/gDW x Irradiance x chla*
# This output will be the biomass normalized photon absorption rate

# read in the chla* values
a_star = pd.read_csv('PBR_optical_abs_xsec.csv',index_col=0,header=0) #experimental values of chla* obtained by Jallet et. al. 2016 (http://dx.doi.org/10.1016/j.algal.2016.05.014)   
a_star = a_star['876'] #values obtained from Jallet et. al. 2016 (http://dx.doi.org/10.1016/j.algal.2016.05.014)
#print(a_star)

# read in the PBR lamp spectral density
rel = pd.read_excel('PBR_sunlight_rel_area.xlsx',header=0) #here it will be used the average relative spectral distribution of sunlight in Chile
#rel

### B3- Define simulation parameters in the photobioreactor

The simulation parameters will be defined as result of experimental setup of microalgae cultivation in PBR. The standard time interval used was the average monthly sunshine hours when microalgae were cultivated (Feb 2016). 

#### <u>Cultivation parameters</u>:

 Time interval: 1 daylight (13.5 horas, 810 min, february 2016, Concepcion, Chile)

 Initial dry weight of 800 L = 90 g DW

 Culture dimensions: cylinder PBR, 183 cm height, diameter= 0.45 cm; total volume ~200L each PBR, total 4 PBR

 Surface area = 2.75 m2

 Maintenance energy = 60 nmol O2 (mg Chla h)-1 #value in Broddrick, 2019

 Simulated temperature: 17.1 ºC

 Cultivation time: 14 days

 Optimal temperature: 20 ºC

#### Output: Initial Biomass composition

In [9]:
# Initialize all initial conditions, output variables and storage pools 
iDW = 90 # initial culture biomass in mg DW (day 1 of cultivation)
xsec = 553 #culture cross sectional area in cm2 #cross section of a tube * 4

rv = 800. # total volume of PBR in L

# make a dict or dataframe that holds the initial gDW of each biomass component
# Pull ratios out of the BOF and multiply them by the total inital mass

rxn = model.reactions.bof_c
data_dict = dict()
for met,s in rxn.metabolites.items():
    if s<0:
        data_dict[met.id]=-1*s*iDW #Takes the intial biomass and calculates the mgDW for each component
data_dict[model.metabolites.carbon_storage_c.id]=0.006*iDW    
data_dict['total']=sum(data_dict.values())
biomass = pd.DataFrame(data=data_dict.values(),index = data_dict.keys(),columns=['initial']) #biomass dataframe
biomass.index.name = 'MET' #add name to the metabolites column
v=data_dict['total'] #set the value of biomass to normalize the biomass dataframe to be integrate later
total_del= biomass.drop(index=['total']) #remove the total at the column (index)
biomass_norm=total_del.div(v) #final biomass normalized dataframe

In [10]:
#_________________________Parameters to be updated each simulation______________________________________________

time_interval = 810 # in  min/day (feb 2016, 1 day 13.5 hours, x60 min, 810 min per day )
days_factor = 14 #14 days of cultivation 
temperature_factor = 0.96 
R = 870. #the average monthly radiation in umol photons m2/s

In [11]:
##________________________Initialize all the parameters and variables for the simulation________________________

# Simulate biomass accumulation

import math as m

iDW = biomass['initial']['total']

gDW = iDW

##________________________Initialize carbon accumulation________________________________________________________
# initial_carbon 
# equal to the inital dry weight biomass component ratios

initial_carbon = 0
biomass_dict = biomass['initial'].to_dict()
for bm,val in biomass_dict.items():
    if bm in model.metabolites:
        met = model.metabolites.get_by_id(bm)
        carbon_mol = met.elements['C']
        carbon_mass = carbon_mol*elements_and_molecular_weights['C']
        initial_carbon = initial_carbon + carbon_mass*val*1000 # multiply by 1000 to obtain biomass in grams 
TOC_counter = dict()
TOC_counter[0]=initial_carbon

model.reactions.bof_c.upper_bound = 0.

#___________________________Initialize variables to collect output data_________________________________________
# Initialize an unused photon count
photon_usage = pd.DataFrame(data=[0,0],index = ['Delivered','Absorbed'],columns=['0'])

# Initialize the carbon storage tracker
mgChry = biomass['initial']['carbon_storage_c']*model.reactions.biomass_c_storage_c.metabolites[model.metabolites.get_by_id('13glucan_c')]*model.metabolites.get_by_id('13glucan_c').formula_weight/1000.*-1
mgTAG = biomass['initial']['carbon_storage_c']*model.reactions.biomass_c_storage_c.metabolites[model.metabolites.tag1619Z1619Z160_c]*model.metabolites.tag1619Z1619Z160_c.formula_weight/1000.*-1
storage_DF = pd.DataFrame(data=[mgChry,mgTAG],index = ['Chrysolaminarin','TAG'],columns=['0'])

# Initialize an O2 evolution rate tracker
o2_evolution_rate = pd.DataFrame(data=[0],index = ['Max oxygen evolution'],columns=['0'])
co2_uptake_rate = pd.DataFrame(data=[0],index = ['Ci uptake'],columns=['0'])
o2_check = pd.DataFrame(data=[0],index = ['Max oxygen evolution'],columns=['0'])

#print(storage_DF)

In [12]:
#____________________________Update the BOF with initial biomass concentration_________________________________

#this script integrates the initial biomass concentration (iDW) into the BOF to initialize the cultivation period.
#The integration is done into the reaction bof_c_accumlation in order to obtain the accumulation carbon over the culture time.

bof_ratios = biomass_norm.to_records(index=True)

bof_tmp = dict()

for m,s in bof_ratios:
    met = model.metabolites.get_by_id(m)
    bof_tmp[met]=s*-1

# copy the bof accumulation reaction and change to carbon storage ratios
r = model.reactions.bof_c_accumulation_c
new_stoich = bof_tmp

# now have a dictionary of new stoichs for the model
for m,s in r.metabolites.items():
    stoich = s*-1
    temp_dict = {m:stoich}
    r.add_metabolites(temp_dict)
    r.add_metabolites(new_stoich)
    
# Then get the total to equal 1 mg biomass DW
total = 0
for m,s in r.metabolites.items():
    gfw = model.metabolites.get_by_id(m.id).formula_weight
    mass = gfw*s*-1
    total = total+mass
correction = total/1000 # this will get it to 1000 ug total mass

# Then adjust the stoichiometry as appropriate
for m,s in r.metabolites.items(): # now change the stoich
    to_add = s/correction-s
    r.add_metabolites({m:to_add})

# Finally build the biomass_c metabolite
imbal = r.check_mass_balance()
if 'charge' in imbal.keys():
    met_charge = imbal['charge']*-1
    del imbal['charge']
met_mass = 0
formula_string = ''
for e,v in imbal.items():
    if v > 1e-12 or v < -1e-12:
        mass = elements_and_molecular_weights[e]
        met_mass = met_mass+(mass*-1*v)
        form_str = e+str(-1*v)
        formula_string = formula_string + form_str

# make a new biomass metabolite
met = model.metabolites.biomass_c_acc_c
met.formula = formula_string
met.charge = met_charge
r.add_metabolites({met:1})

# Add GAM constraint
gam_met = model.metabolites.GAM_const_c
r.add_metabolites({gam_met:1})

model.repair()

### B4 - Light constraints                                            

The following script calculate the incident light capturing. It uses the incident light equation (Platt equation) to caculate the amount of light inside the PBR in the middle of the time interval, according the following:

                                PP = Ps(1-eE(-alpha*I/Ps)eE(-beta*I/Ps))

where Ps is the maximum photosynthetic rate. Since the Platt equation describes a hyperbolic relationship between photosynthesis and irradiance, the parameter α represents the initial slope of the hyperbolic curve of photosynthesis at low light intensity.


In [13]:
#Initial dataframes and unit convertions

import math as m

tmp_data = dict(zip(data_dict.keys(),[0,0,0,0,0,0,0,0,0])) #Initialize the biomass accumulator
del tmp_data['total']
interval_bm = 0 #initializes the total biomass
photon_count = 0 
atten = np.zeros(len(a_star)) #captures light attenuation
        
tmp_o2_evo = 0 #oxygen evolution as photosynthetic rate in this study
        
gDW = biomass['initial']['total'] 
        
chla_stoich = model.reactions.biomass_pigm_h.metabolites[model.metabolites.cholphya_h] # umol Chla per mg pigments
chla = chla_stoich*model.metabolites.cholphya_h.formula_weight*-0.001 # converts to mg Chla per mg pigments
mgchla = chla *(biomass['initial']['biomass_pigm_h'])# mg Chla


#O2 rate equation Returns the parameters to get the O2 rate per umol photons absorbed/mgChla*s

alpha = 147.3 #as the original publication
beta = -2.86  #as the original publication
Ps = ((-6.0869e-4)*m.pow((time_interval/2.),2))+(4.3142e-1*(time_interval/2.))+296.2
irrad = (R)*(m.sin(2.*m.pi*(1.1574074074074e-5)*(time_interval/2.)*60.)) # modeled per hour
# incident light equation set to the middle of the time interval
# in umol photons/m2*s

photon_flux = (rel['rel_area'].values)*irrad*xsec/10000*time_interval*60. #10000 for converting cm2 to m2
total_photons = sum(photon_flux)
photon_usage=[total_photons,0] 

# Walk through the spectrum calculating delivered and absorbed photons 

corr_a_star = a_star*1.2  #1.2 is the experimental volume for O2 evolution rate mensurements in Broddrick (2019)
        
ti_o2 = 0.

# First get delivered photons
delivered_photons = photon_flux

# Variable to collect the photons absorbed
total_absorbed = 0.
o2 = 0. 

for nm in range(len(photon_flux)):
    abs_coeff = corr_a_star[400+nm]                  # a* value for the given nm (m2/mgchla)
    nm_delivered = delivered_photons[nm]             # incident photon flux at this nm (umol_photon/time_interval)
    nm_abs = nm_delivered*abs_coeff*(mgchla*rv)/xsec*10000 # photons absorbed at this nm at this slice(umol/TI * m2/mgchla * mgchla * 1/cm2 * 10000cm2/m2) 

# Update the photon_flux variable at this wavelength
    new_delivered = nm_delivered - nm_abs
    if new_delivered >= 0:
        photon_flux[nm] = new_delivered
    else:
        photon_flux[nm] = 0.

# Append the running photon absorption
total_absorbed = total_absorbed + nm_abs

#Update the variables that will constrain photon flux
              
conv_abs = total_absorbed/time_interval/60./(mgchla*rv) # converts abs to O2 evo curve units  umol/TI/s/mgchla*mL => umol/(mgchla*s) 
o2 = Ps*(1-m.exp(-1*(alpha*conv_abs)/Ps))*(m.exp(-1*(beta*conv_abs)/Ps)) #O2 evolution umolO2/(mgChla*h) 
       
ti_o2 = ti_o2+(o2*(rv*mgchla)/60.*time_interval) #O2 evolution rate per day (hours of light)

o2_check=ti_o2
abs_units = total_absorbed/60./(mgchla) # converts abs to O2 evo curve units  umol/TI * TI/min * min/s * 1/mgchla => umol/(mgchla*s) 
o2evo = ti_o2
model.reactions.EX_o2_e.upper_bound = o2evo

    
                
#Specific constraints

ngam = 3600./(mgchla * time_interval) #ngam = (60 umol O2) / (mg Chla * time_interval /60)

model.reactions.AOX_m.upper_bound = 1000. #upper_bound > lower_bound
model.reactions.AOX_m.lower_bound = ngam
  
#cef constraints
cef_rate = 5.*gDW/60.*time_interval #CEF sets CEF_h upper bound.  umol /(mgDW*h) * mgDW * h/60min * min/TI = umol/TI
model.reactions.CEF_h.upper_bound = cef_rate  

In [14]:
###__________________________________OBJECTIVE FUNCTION____________________________________

model.objective = 'bof_c_accumulation_c'
solution = model.optimize()
if solution.status == 'optimal':
    obj_rxn = model.reactions.bof_c_accumulation_c
    
    for bm,val in tmp_data.items():
        bm_flux = obj_rxn.x

        if bm in model.metabolites and model.metabolites.get_by_id(bm) in obj_rxn.metabolites.keys():
            stoich = obj_rxn.metabolites[model.metabolites.get_by_id(bm)]
            tmp_flux = stoich*bm_flux*-1.
            total_flux = val+tmp_flux
            tmp_data[bm]=total_flux

        else:
            tmp_data[bm]=0.

        interval_bm = sum(tmp_data.values())

    else: 
        interval_bm = interval_bm + 0.
                
    tmp_data['total']=interval_bm

    
    biomass['14 days'] = (pd.Series(tmp_data)*days_factor*temperature_factor) #add a colum with the tmp_data values to biomass dataframe
    #biomass['sum'] = biomass['initial']+biomass['14 days']
    interval_totals = biomass['14 days']
    
    biomass_total= interval_totals   
    
    mgChry = biomass_total['carbon_storage_c']*model.reactions.biomass_c_storage_c.metabolites[model.metabolites.get_by_id('13glucan_c')]*model.metabolites.get_by_id('13glucan_c').formula_weight/1000.*-1
    mgTAG = biomass_total['carbon_storage_c']*model.reactions.biomass_c_storage_c.metabolites[model.metabolites.tag1619Z1619Z160_c]*model.metabolites.tag1619Z1619Z160_c.formula_weight/1000.*-1
    storage_DF= [mgChry,mgTAG]
        
    interval_TOC = 0
    carbon_count = biomass_total.to_dict()
    for bm,val in carbon_count.items():
        if bm in model.metabolites:
            met = model.metabolites.get_by_id(bm)
            carbon_mol = met.elements['C']
            carbon_mass = carbon_mol*elements_and_molecular_weights['C']
            interval_TOC = interval_TOC + carbon_mass*val
           
    TOC_counter=interval_TOC
    o2_evolution_rate=tmp_o2_evo
        
         
photon_usage=total_absorbed

           
      
# biomass
# print(solution)

In [15]:
biomass #results in g/L

,initial,14 days
MET,,
biomass_carb_c,14.110589,30.606860
biomass_dna_c,0.737926,1.600614
biomass_mem_lipids_c,9.620038,20.866539
biomass_pigm_h,7.487251,16.240375
biomass_pro_c,52.167783,113.155593
biomass_rna_c,5.876412,12.746352
carbon_storage_c,0.540000,1.171298
total,90.540000,196.387631
